# T1070.003 - Clear Command History
In addition to clearing system logs, an adversary may clear the command history of a compromised account to conceal the actions undertaken during an intrusion. Various command interpreters keep track of the commands users type in their terminal so that users can retrace what they've done.

On Linux and macOS, these command histories can be accessed in a few different ways. While logged in, this command history is tracked in a file pointed to by the environment variable <code>HISTFILE</code>. When a user logs off a system, this information is flushed to a file in the user's home directory called <code>~/.bash_history</code>. The benefit of this is that it allows users to go back to commands they've used before in different sessions.

Adversaries may delete their commands from these logs by manually clearing the history (<code>history -c</code>) or deleting the bash history file <code>rm ~/.bash_history</code>.  

Adversaries may also leverage a [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) on network devices to clear command history data (<code>clear logging</code> and/or <code>clear history</code>).(Citation: US-CERT-TA18-106A)

On Windows hosts, PowerShell has two different command history providers: the built-in history and the command history managed by the <code>PSReadLine</code> module. The built-in history only tracks the commands used in the current session. This command history is not available to other sessions and is deleted when the session ends.

The <code>PSReadLine</code> command history tracks the commands used in all PowerShell sessions and writes them to a file (<code>$env:APPDATA\Microsoft\Windows\PowerShell\PSReadLine\ConsoleHost_history.txt</code> by default). This history file is available to all sessions and contains all past history since the file is not deleted when the session ends.(Citation: Microsoft PowerShell Command History)

Adversaries may run the PowerShell command <code>Clear-History</code> to flush the entire command history from a current PowerShell session. This, however, will not delete/flush the <code>ConsoleHost_history.txt</code> file. Adversaries may also delete the <code>ConsoleHost_history.txt</code> file or edit its contents to hide PowerShell commands they have run.(Citation: Sophos PowerShell command audit)(Citation: Sophos PowerShell Command History Forensics)

## Atomic Tests

### Atomic Test #1 - Clear Bash history (rm)Clears bash history via rm
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
rm ~/.bash_history
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 1

### Atomic Test #2 - Clear Bash history (echo)Clears bash history via echo
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
echo "" > ~/.bash_history
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 2

### Atomic Test #3 - Clear Bash history (cat dev/null)Clears bash history via cat /dev/null
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
cat /dev/null > ~/.bash_history
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 3

### Atomic Test #4 - Clear Bash history (ln dev/null)Clears bash history via a symlink to /dev/null
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
ln -sf /dev/null ~/.bash_history
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 4

### Atomic Test #5 - Clear Bash history (truncate)Clears bash history via truncate
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
truncate -s0 ~/.bash_history
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 5

### Atomic Test #6 - Clear history of a bunch of shellsClears the history of a bunch of different shell types by setting the history size to zero
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
unset HISTFILE
export HISTFILESIZE=0
history -c
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 6

### Atomic Test #7 - Clear and Disable Bash History LoggingClears the history and disable bash history logging of the current shell and future shell sessions
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
set +o history
echo 'set +o history' >> ~/.bashrc
. ~/.bashrc
history -c
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 7

#### Cleanup: 
```sh
sed -i 's/set +o history//g' ~/.bashrc
. ~/.bashrc
set -o history
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Use Space Before Command to Avoid Logging to HistoryUsing a space before a command causes the command to not be logged in the Bash History file
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
hostname
whoami
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 8

### Atomic Test #9 - Disable Bash History Logging with SSH -T
Keeps history clear and stays out of lastlog,wtmp,btmp ssh -T keeps the ssh client from catching a proper TTY, which is what usually gets logged on lastlog

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: Install sshpass and create user account used for excuting

##### Check Prereq Commands:
```sh
$(getent passwd testuser1 >/dev/null) && $(which sshpass >/dev/null)

```
##### Get Prereq Commands:
```sh
[ "$(uname)" = 'FreeBSD' ] && pw useradd testuser1 -g wheel -s /bin/sh || /usr/sbin/useradd testuser1
[ "$(uname)" = 'FreeBSD' ] && echo 'pwd101!' | pw mod user testuser1 -h 0 || echo -e 'pwd101!\npwd101!' | passwd testuser1
(which yum && yum -y install epel-release sshpass)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y sshpass)||(which pkg && pkg install -y sshpass)

```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sshpass -p 'pwd101!' ssh testuser1@localhost -T hostname
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 9

#### Cleanup: 
```sh
[ "$(uname)" = 'FreeBSD' ] && rmuser -y testuser1 || userdel -f testuser1
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Prevent Powershell History LoggingPrevents Powershell history
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Set-PSReadlineOption -HistorySaveStyle SaveNothing
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 10

#### Cleanup: 
```powershell
Set-PSReadLineOption -HistorySaveStyle SaveIncrementally```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Clear Powershell History by Deleting History FileClears Powershell history
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Remove-Item (Get-PSReadlineOption).HistorySavePath
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 11

### Atomic Test #12 - Set Custom AddToHistoryHandler to Avoid History File LoggingThe "AddToHistoryHandler" receives the current command as the $line variable and then returns $true if 
the line should be written to the history file. Here we simply return $false so nothing gets added to 
the history file for the current session. 
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Set-PSReadLineOption -AddToHistoryHandler { return $false }
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 12

#### Cleanup: 
```powershell
Set-PSReadLineOption -AddToHistoryHandler $null
```

In [ ]:
Invoke-AtomicTest T1070.003 -TestNumbers 12 -Cleanup

## Detection
User authentication, especially via remote terminal services like SSH, without new entries in that user's <code>~/.bash_history</code> is suspicious. Additionally, the removal/clearing of the <code>~/.bash_history</code> file can be an indicator of suspicious activity.

Monitor for suspicious modifications or deletion of <code>ConsoleHost_history.txt</code> and use of the <code>Clear-History</code> command.